In [ ]:
import requests
import re
import json
# from findw_spiders.utils.funcs import *
import pandas as pd
import time
# import findw3.api.clickhouse as ch
# from findw3.utils.email import send_mail
# from config import MAX_TRY_COUNT,TIME_OUT,COOKIE_IP
# from findw3.utils.ip_pool import get_ip
from queue import Queue
import datetime
# from findw_spiders.utils.funcs import csv_append
import threading
import os
# 要改成cookies过期时停止进程

# import redis
import time
import datetime
import json
import requests
# from findw3.utils.email import send_mail
import json


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import matplotlib.pyplot as plt
import random
# from findw3.utils.spider import open_chrome, close_chrome#, csv_append
# from findw_spiders.utils.funcs import open_chrome, close_chrome, csv_append

from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from PIL import Image 
import base64
from datetime import datetime

import os
from selenium.webdriver.chrome.options import Options

In [ ]:
def inputall(wait, phone_number, password):
    try:
        try:
            login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-analytics='14']")))
            login_button.click()
            print("切换到用户名登录成功。")
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print("切换到用户名登录失败。")
            raise  # 重新抛出异常，以便上层处理
        time.sleep(1)
        try:
            phone_input = wait.until(EC.presence_of_element_located((By.NAME, "username")))
            phone_input.send_keys(phone_number)
            print("输入账号成功。")
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print("输入账号失败。")
            raise
        time.sleep(1)
        try:
            password_input = wait.until(EC.presence_of_element_located((By.NAME, "password")))
            password_input.send_keys(password)
            print("输入密码成功。")
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print("输入密码失败。")
            raise

        time.sleep(2)
        try:
            checkbox = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".iconfont_iconfont_9UW.newLogin_nochecked_3WZ")))
            checkbox.click()
            print("点击统一隐私策略成功。")
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print("点击统一隐私策略失败。")
            raise
        time.sleep(1)

        try:
            login_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal__login__main"]/div[2]/div[2]')))
            login_button.click()
            print("点击登录按钮成功。")
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            print("点击登录按钮失败。")
            raise
    except Exception as e:
        print(f"登录过程中发生未预料的异常：{e}")
        raise
        
def save_image(browser, wait, selector, path):
    try:
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
        class_name = element.get_attribute("class")
        js_cmd = (
            'return document.getElementsByClassName("%s")[0].toDataURL("image/png");'
            % class_name
        )
        im_info = browser.execute_script(js_cmd)
        im_base64 = im_info.split(",")[1]
        im_bytes = base64.b64decode(im_base64)
        with open(path, "wb") as f:
            f.write(im_bytes)
    except Exception as e:
        raise RuntimeError(f"保存验证码图像失败 {selector}") from e

def save_images(browser, wait):
    selectors_paths = [
        ("body > div.geetest_panel.geetest_wind > div.geetest_panel_box.geetest_no_logo.geetest_panelshowslide > div.geetest_panel_next > div > div.geetest_wrap > div.geetest_widget > div > a > div.geetest_canvas_img.geetest_absolute > div > canvas.geetest_canvas_bg.geetest_absolute", './background.png'),
        ("body > div.geetest_panel.geetest_wind > div.geetest_panel_box.geetest_no_logo.geetest_panelshowslide > div.geetest_panel_next > div > div.geetest_wrap > div.geetest_widget > div > a > div.geetest_canvas_img.geetest_absolute > div > canvas.geetest_canvas_slice.geetest_absolute", './slider.png'),
        ("body > div.geetest_panel.geetest_wind > div.geetest_panel_box.geetest_no_logo.geetest_panelshowslide > div.geetest_panel_next > div > div.geetest_wrap > div.geetest_widget > div > a > div.geetest_canvas_img.geetest_absolute > canvas", './full_background.png')
    ]

    for selector, path in selectors_paths:
        save_image(browser, wait, selector, path)
        
def is_pixel_equal(image1, image2, x, y):
    """
    判断两个像素是否相同
    :param image1: 图片 1
    :param image2: 图片 2
    :param x: 位置 x
    :param y: 位置 y
    :return: 像素是否相同
    """
    # 取两个图片的像素点
    pixel1 = image1.load()[x, y]
    pixel2 = image2.load()[x, y]
    threshold = 60
    if abs(pixel1[0] - pixel2[0]) < threshold and abs(pixel1[1] - pixel2[1]) < threshold and abs(pixel1[2] - pixel2[2]) < threshold:
        return True
    else:
        return False
# 计算带缺口背景图与全背景图的差距 ，得出缺口的offset
def get_gap( image1, image2):
    """
    获取缺口偏移量
    :param image1: 不带缺口图片
    :param image2: 带缺口图片
    :return: 缺口位置
    """
    left = 10
    for i in range(left, image1.size[0]):
        for j in range(image1.size[1]):
            if not is_pixel_equal(image1, image2, i, j):
                left = i
                return left
    return left


def easeOutBounce(t, b, c, d):
    t /= d
    if t < 1 / 2.75:
        return c * (7.5625 * t * t) + b
    elif t < 2 / 2.75:
        t -= 1.5 / 2.75
        return c * (7.5625 * t * t + 0.75) + b
    elif t < 2.5 / 2.75:
        t -= 2.25 / 2.75
        return c * (7.5625 * t * t + 0.9375) + b
    else:
        t -= 2.625 / 2.75
        return c * (7.5625 * t * t + 0.984375) + b

def get_tracks(distance, duration=5):
    tracks = []
    current = 0
    t = 0.0
    while current < distance:
        move = easeOutBounce(t, 0, distance, duration)
        move = round(move - current)
        current += move
        tracks.append(move)
        t += 0.1
    return tracks

# 移动滑块
def move_to_gap(browser, wait, tracks):
    """滑动滑块"""
    slider = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'geetest_slider_button')))
    ActionChains(browser).click_and_hold(slider).perform()
    time.sleep(0.01)
    for x in tracks:
        ActionChains(browser).move_by_offset(xoffset=x, yoffset=0).perform()
        time.sleep(random.uniform(0.01, 0.03))
    time.sleep(1)
    ActionChains(browser).release().perform()

# 使用pyautogui滑动的，但是成功率感觉差不多，只是肉眼看上去似乎好些了
# def move_to_gap(tracks):
#     slider = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'geetest_slider_button')))
#     location = slider.location
#     size = slider.size
#     start_x = location['x']+40# + size['width'] / 2
#     start_y = location['y']+200 #+ size['height'] / 2
#     pyautogui.moveTo(start_x-40, start_y-200)
#     pyautogui.mouseDown()
#     pyautogui.moveTo(start_x, start_y)
#     time.sleep(0.2)
#     pyautogui.mouseDown()
#     time.sleep(0.2)
#     for x in tracks:
#         pyautogui.moveRel(x, 0, duration=random.uniform(0.001, 0.003))
#     pyautogui.mouseUp()
def perform_verification(browser, wait):
    while True:
        save_images(browser, wait)
        full_bg = Image.open('full_background.png')
        bg = Image.open('background.png')
        distance = get_gap(bg, full_bg)
        tracks = get_tracks(distance - 5)
        move_to_gap(browser, wait, tracks)
        time.sleep(5)
        cookies = browser.get_cookies()
        for cookie in cookies:
            if cookie.get('name') == 'remember':
                print('验证成功')
                return
        time.sleep(5)
        error_content = browser.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[4]/div[3]")
        error_content.click()
        print('验证失败')
     
            

def new_cookies(phone_number, password):
    
    try:
        cookies = []
        # close_chrome()
#         custom_profile_path = '/tmp/chrome_profile'  # 使用/tmp目录
#         custom_profile_path = os.path.join(os.path.expanduser('~'), 'chrome_profile')
#         options = Options()
#         options.add_argument("--user-data-dir=" + custom_profile_path)
        s = Service('/usr/local/bin/chromedriver')
        browser = webdriver.Chrome(service=s)
        browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
                })
            """
        })
        wait = WebDriverWait(browser, 20)
        browser.get('https://xueqiu.com')
        inputall(wait, phone_number, password)
#         browser = webdriver.Chrome()
        perform_verification(browser, wait)
        cookies = browser.get_cookies()
    except TimeoutException:
        print("操作超时，请检查网络连接或页面元素是否正确加载。")
    except NoSuchElementException as e:
        print(f"未找到元素：{e}。请检查页面元素是否正确或是否已经加载。")
    except WebDriverException as e:
        print(f"WebDriver异常：{e}。请检查WebDriver配置或浏览器驱动是否正常工作。")
    except Exception as e:
        print(f"发生未知错误：{e}。")
    finally:
        # close_chrome()
        browser.quit()

    return cookies

def update_cookies(phone_number, password):
    while True:
        cookies = new_cookies(phone_number, password)
        remember_cookie_found = any(cookie.get('name') == 'remember' for cookie in cookies) 
        if remember_cookie_found:
            now = datetime.now()
            success_message = f"成功: {now} - 'remember' cookie 已找到。"
            break
        else:
            now = datetime.now()
            failure_message =f"失败:{now} 'remember' cookie 未找到。"
            
    return cookies 

def cookies_to_string(cookies):
    cookie_list = []
    for cookie in cookies:
        cookie_str = f"{cookie['name']}={cookie['value']}"
        cookie_list.append(cookie_str)
    cookies_string = '; '.join(cookie_list)

    return cookies_string


def init_cookies():
    update_rows = []
    for value in ACCOUNT.values():
        current_time = datetime.now()
        phone_number = value["phone_number"]
        password =  value["password"]
        new_cookie = update_cookies(phone_number, password)
        new_cookie = cookies_to_string(new_cookie)
        new_row = {
                'user_id': phone_number,
                't_date': current_time.strftime('%Y-%m-%d'),
                's_date': current_time.strftime('%Y-%m-%d'),
                'e_date': pd.NaT,  # 使用pd.NaT表示NaN日期
                # ... 其他列 ...
                'cookie': new_cookie,
                'update_time': current_time.strftime('%Y-%m-%d %H:%M:%S')
            }
        update_rows.append(new_row)
    df = pd.DataFrame(update_rows)   
    df.to_csv('./cookies.csv')
    return df
ACCOUNT = {
    "1": {"phone_number": "18583930319", "password": "328800Hjz"},
    # "2": {"phone_number": "18205536444", "password": "Pi31415926"},
    "3": {"phone_number": "13651009080", "password": "00cb07775df9"},
    # "4": {"phone_number": "18519031092", "password": "E2718281828"},
    # "5": {"phone_number": "13716511196", "password": "E2718281828"},
    # "6": {"phone_number": "18600044699", "password": "Pi31415926"},
    # "7": {"phone_number": "13382781939", "password": "1234567890"},
    # "8": {"phone_number": "18256561019", "password": "123456"},
    "9": {"phone_number": "13723286312", "password": "E2718281828"},
    "10": {"phone_number": "13781154338", "password": "a"},
}